# Finding Cheapest To Deliver in a Futures Contract

In [2]:
import pandas as pd
import QuantLib as ql
from MachineTrading.Instruments.fixed_rate import FixedRateBond

Read Actual EUREX data for date 2022-04-01

In [3]:
mts_data = pd.read_csv(r"JupyterResources/deliverable-2022-04-01.csv", sep=';')

In [12]:
deliverable = mts_data[mts_data['#Contract']=='FBTP JUN 2022'].copy()
deliverable

,#Contract,ISIN,Coupon,Maturity,ConvFac
44,FBTP JUN 2022,IT0005422891,0.90,01.04.2031,0.659620
45,FBTP JUN 2022,IT0005436693,0.60,01.08.2031,0.628867
46,FBTP JUN 2022,IT0005449969,0.95,01.12.2031,0.643888
47,FBTP JUN 2022,IT0005466013,0.95,01.06.2032,0.630012


In [8]:
date = ql.Date(1, 4, 2022)

# Set the basket of deliverables:
basket = [(0.90, ql.Date(1, 4, 2031), ql.Date(1, 10, 2020), 90.81), # IT0005422891  ->  0.659620
          (0.60, ql.Date(1, 8, 2031), ql.Date(23, 2, 2021), 88.5),  # IT0005436693  ->  0.628867
          (0.95, ql.Date(1, 12, 2031), ql.Date(1, 6, 2021), 91.0),  # IT0005449969  ->  0.643888
          (0.95, ql.Date(1, 6, 2032), ql.Date(1, 11, 2021), 90.09)  # IT0005466013  ->  0.630012
          ]

f_price = 137.54  # FBTP Settlement Price
f_delivery = ql.Date(10,6,2022)  # Delivery date for FBTP

securities = []
min_basis = 100
min_basis_index = -1
for i, b in enumerate(basket):
    coupon, maturity, issue, price = b
    
    # Set the bond: 
    s = FixedRateBond(issue, maturity, coupon, settlement_days=0, calendar=ql.Germany())
    
    # Compute conversion factors and basis
    cf, basis = s.futures_contract_conversion_factor(bond_price=price, futures_price=f_price,
                                                     futures_delivery_date=f_delivery)
    # Check for CTD
    if basis < min_basis:
        min_basis = basis
        min_basis_index = i
    securities.append((s, cf, basis))

ctd_info = basket[min_basis_index]
ctd_bond, ctd_cf, ctd_basis = securities[min_basis_index]
ctd_price = ctd_info[3]

print("%-30s = %lf" % ("Minimum Basis", min_basis))
print("%-30s = %lf" % ("Conversion Factor", ctd_cf))
print("%-30s = %lf" % ("Coupon", ctd_info[0]))
print("%-30s = %s" % ("Maturity", ctd_info[1]))
print("%-30s = %lf" % ("Price", ctd_info[3]))


Minimum Basis                  = 0.085862
Conversion Factor              = 0.659620
Coupon                         = 0.900000
Maturity                       = April 1st, 2031
Price                          = 90.810000


In [11]:
[round(x[1],6) for x in securities]

[0.65962, 0.628867, 0.643888, 0.630012]

In [14]:
deliverable['ComputedConvFac'] = [round(x[1],6) for x in securities]
deliverable['ComputedBasis'] = [round(x[2],6) for x in securities]

In [15]:
deliverable

,#Contract,ISIN,Coupon,Maturity,ConvFac,ComputedConvFac,ComputedBasis
44,FBTP JUN 2022,IT0005422891,0.90,01.04.2031,0.659620,0.659620,0.085862
45,FBTP JUN 2022,IT0005436693,0.60,01.08.2031,0.628867,0.628867,2.005626
46,FBTP JUN 2022,IT0005449969,0.95,01.12.2031,0.643888,0.643888,2.439676
47,FBTP JUN 2022,IT0005466013,0.95,01.06.2032,0.630012,0.630012,3.438088
